# Examples

## Fetch KODI data

In this example, we fetch data from the Kenya Open Data Initiative API.

In [1]:
from riko.modules.fetchdata import pipe
poverty_url = 'opendata.go.ke/resource/i5bp-z9aq.json'
stream = pipe(conf={'url': poverty_url})
next(stream)

{'district_name': 'Turkana',
 'number_of_poor_2005_06_': '481442',
 'poverty_rate_2005_06_': '94.3'}

## Summing

Now we will sum the total number of poor in Kenya

In [ ]:
from riko.collections.sync import SyncPipe

sum_conf = {'sum_key': 'number_of_poor_2005_06_'}
stream = SyncPipe('fetchdata', conf={'url': poverty_url}).sum(conf=sum_conf).output
"{:,}".format(next(stream)['sum'])

## Sorting

In [6]:
sort_conf = {'rule': {'sort_key': 'poverty_rate_2005_06_'}}
stream = SyncPipe('fetchdata', conf={'url': poverty_url}).sort(conf=sort_conf).output
next(stream)

{'district_name': 'Kajiado',
 'number_of_poor_2005_06_': '46578',
 'poverty_rate_2005_06_': '11.6'}

## Filtering and truncating

Now, we will filter the data for districts with more than 70% poverty and list the top 5.

In [49]:
sort_conf = {'rule': {'sort_key': 'poverty_rate_2005_06_', 'sort_dir': 'desc'}}
filter_conf = {'rule': {'field': 'poverty_rate_2005_06_', 'op': 'greater', 'value': 70}}
stream = (
    SyncPipe('fetchdata', conf={'url': poverty_url})
        .filter(conf=filter_conf)
        .sort(conf=sort_conf)
        .truncate(conf={'count': '5'})
        .output)

list(stream)

[{'district_name': 'Turkana',
  'number_of_poor_2005_06_': '481442',
  'poverty_rate_2005_06_': '94.3'},
 {'district_name': 'Marsabit',
  'number_of_poor_2005_06_': '118786',
  'poverty_rate_2005_06_': '91.7'},
 {'district_name': 'Mandera',
  'number_of_poor_2005_06_': '225812',
  'poverty_rate_2005_06_': '87.8'},
 {'district_name': 'Wajir',
  'number_of_poor_2005_06_': '301422',
  'poverty_rate_2005_06_': '84.0'},
 {'district_name': 'Tana River',
  'number_of_poor_2005_06_': '191856',
  'poverty_rate_2005_06_': '76.9'}]

## Counting

Now we will count the total number of schools in Kenya

In [2]:
# First let's look at the data
from riko.modules import fetchdata, join

schools_url = 'opendata.go.ke/resource/fbd2-c7tq.json'
next(fetchdata.pipe(conf={'url': schools_url}))


{'boys_toilets': '1',
 'costituency': 'KABETE',
 'county': 'KIAMBU',
 'district': 'KIAMBU',
 'division': 'KIKUYU',
 'geolocation': {'human_address': '{"address":"","city":"","state":"","zip":""}',
  'latitude': '-1.25109',
  'longitude': '36.69185',
  'needs_recoding': False},
 'girls_toilets': '3',
 'gok_tsc_female': '0',
 'gok_tsc_male': '0',
 'level_of_education': 'PRIMARY SCHOOL',
 'local_authority_female': '0',
 'local_authority_male': '0',
 'location': 'KINOO',
 'name_of_school': 'KAGA ACADEMY PRIMARY',
 'non_teaching_staff_female': '0',
 'non_teaching_staff_male': '0',
 'others_female': '0',
 'others_male': '1',
 'province': 'CENTRAL',
 'pta_bog_female': '0',
 'pta_bog_male': '0',
 'pupil_classroom_ratio': '23',
 'pupil_teacher_ratio': '69',
 'pupil_toilet_ratio': '17.3',
 'school_institution_type_1': 'MIXED',
 'school_institution_type_2': 'BOARDING ONLY',
 'school_institution_type_3': 'INTEGRATED',
 'sponsor_of_school': 'PRIVATE INDIVIDUAL',
 'status_of_school': 'PRIVATE',
 'te

In [7]:
# Now let's count schools by division
from riko.collections.sync import SyncPipe

count_conf = {'count_key': 'division'}
stream = SyncPipe('fetchdata', conf={'url': schools_url}).count(conf=count_conf).output
list(stream)

[{'ASEGO': 1},
 {'BUTERE': 1},
 {'DAGORETTI': 2},
 {'EMBAKASI': 1},
 {'ISLAND': 1},
 {'KANDARA': 1},
 {'KASIKEU': 1},
 {'KIBERA': 2},
 {'KIKUYU': 1},
 {'KISAUNI': 1},
 {'KWANZA': 1},
 {'MARAGUA': 1},
 {'MWEA': 1},
 {'YATTA': 1}]

## More sums

In [11]:
sum_conf = {'sum_key': 'teachers_toilets', 'group_key': 'county'}
stream = SyncPipe('fetchdata', conf={'url': schools_url}).sum(conf=sum_conf).output
list(stream)

[{'EMBU': Decimal('1')},
 {'HOMA BAY': Decimal('2')},
 {'KAKAMEGA': Decimal('1')},
 {'KIAMBU': Decimal('1')},
 {'MACHAKOS': Decimal('6')},
 {'MAKUENI': Decimal('2')},
 {'MOMBASA': Decimal('3')},
 {"MURANG'A": Decimal('17')},
 {'NAIROBI': Decimal('8')},
 {'TRANS NZOIA': Decimal('2')}]

## Joining

Now, we will join Kenya Primary School data to the poverty data.

In [34]:
# HACK: create a helper function to lower case the key name 

def lower_case(stream, key): 
    for item in stream:
        item[key] = item.get(key, '').lower()
        yield item

In [35]:
# Looks good, let's join them now

poverty_stream = fetchdata.pipe(conf={'url': poverty_url})
schools_stream = fetchdata.pipe(conf={'url': schools_url})
join_conf = {'join_key': 'district_name', 'other_join_key': 'district'}

poverty_l_stream = lower_case(poverty_stream, join_conf['join_key'])
schools_l_stream = lower_case(schools_stream, join_conf['other_join_key'])
joined_stream = pipe(poverty_l_stream, conf=join_conf, other=schools_l_stream)
next(joined_stream)

{'boys_toilets': '4',
 'costituency': 'KILOME',
 'county': 'MAKUENI',
 'district': 'makueni',
 'district_name': 'makueni',
 'division': 'KASIKEU',
 'geolocation': {'human_address': '{"address":"","city":"","state":"","zip":""}',
  'latitude': '-2.01791',
  'longitude': '37.38022',
  'needs_recoding': False},
 'girls_toilets': '6',
 'gok_tsc_female': '0',
 'gok_tsc_male': '0',
 'level_of_education': 'PRIMARY SCHOOL',
 'local_authority_female': '0',
 'local_authority_male': '0',
 'location': 'KASIKEU',
 'name_of_school': 'TRAVELLERS OASIS CENTRE',
 'non_teaching_staff_female': '0',
 'non_teaching_staff_male': '0',
 'number_of_poor_2005_06_': '619148',
 'others_female': '0',
 'others_male': '3',
 'poverty_rate_2005_06_': '64.1',
 'province': 'EASTERN',
 'pta_bog_female': '0',
 'pta_bog_male': '0',
 'pupil_classroom_ratio': '2.5',
 'pupil_teacher_ratio': '6.7',
 'pupil_toilet_ratio': '2',
 'school_institution_type_1': 'MIXED',
 'school_institution_type_2': 'BOARDING ONLY',
 'school_institu